# 用于处理多模态RAG图文问答挑战赛训练集JSON文件，提取问题与答案构建QA对
本notebook将演示如何将原始训练集转换为标准QA格式，便于后续微调。

In [1]:
# 1. 导入所需库
import json
import os
import pandas as pd

## 2. 加载JSON数据集
读取多模态RAG图文问答挑战赛训练集.json文件，解析为Python对象。

In [2]:
# 2. 加载JSON数据集
data_path = "spark_multi_rag/datas/多模态RAG图文问答挑战赛训练集.json"
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"样本数量: {len(data)}")
print("示例样本:", data[0])

样本数量: 118
示例样本: {'filename': '联邦制药-港股公司研究报告-创新突破三靶点战略联姻诺和诺德-25071225页.pdf', 'page': 4, 'question': '根据联邦制药（03933.HK）的公司发展历程，请简述其在2023年的重大产品临床进展。', 'answer': '根据联邦制药（03933.HK）的公司发展历程，2023年的重大产品临床进展如下：\n\n- **重磅产品临床**：\n  - UBT251多个适应症进入临床阶段；\n  - 司美格鲁肽注射液体重管理适应症获临床受理；\n  - 利拉鲁肽注射液BLA；德谷胰岛素利拉鲁肽注射液获批临床。'}


## 3. 数据结构与字段解析
分析每条数据的字段结构，重点关注filename、page、question、answer等字段。

In [3]:
# 3. 数据结构与字段解析
sample = data[0]
print("字段列表:", list(sample.keys()))
for k, v in sample.items():
    print(f"{k}: {v}")

字段列表: ['filename', 'page', 'question', 'answer']
filename: 联邦制药-港股公司研究报告-创新突破三靶点战略联姻诺和诺德-25071225页.pdf
page: 4
question: 根据联邦制药（03933.HK）的公司发展历程，请简述其在2023年的重大产品临床进展。
answer: 根据联邦制药（03933.HK）的公司发展历程，2023年的重大产品临床进展如下：

- **重磅产品临床**：
  - UBT251多个适应症进入临床阶段；
  - 司美格鲁肽注射液体重管理适应症获临床受理；
  - 利拉鲁肽注射液BLA；德谷胰岛素利拉鲁肽注射液获批临床。


## 4. 构建QA对列表
遍历数据集，将每条数据的question和answer字段提取出来，构建QA对列表。指令字段统一设置为“你是一名专业的财报数据问答助手”。

In [4]:
# 4. 构建QA对列表
qa_list = []
for item in data:
    qa = {
        "instruction": "你是一名专业的财报数据问答助手",
        "input": item.get("question", ""),
        "output": item.get("answer", "")
    }
    qa_list.append(qa)
print("QA样例:", qa_list[0])
print(f"总QA数量: {len(qa_list)}")

QA样例: {'instruction': '你是一名专业的财报数据问答助手', 'input': '根据联邦制药（03933.HK）的公司发展历程，请简述其在2023年的重大产品临床进展。', 'output': '根据联邦制药（03933.HK）的公司发展历程，2023年的重大产品临床进展如下：\n\n- **重磅产品临床**：\n  - UBT251多个适应症进入临床阶段；\n  - 司美格鲁肽注射液体重管理适应症获临床受理；\n  - 利拉鲁肽注射液BLA；德谷胰岛素利拉鲁肽注射液获批临床。'}
总QA数量: 118


## 5. 保存处理后的QA数据
将处理后的QA对列表保存为新的JSON或CSV文件，便于后续模型训练或分析。

In [5]:
# 5. 保存处理后的QA数据
qa_json_path = "spark_multi_rag/datas/qa_train.json"
with open(qa_json_path, "w", encoding="utf-8") as f:
    json.dump(qa_list, f, ensure_ascii=False, indent=2)
print(f"已保存JSON文件: {qa_json_path}")

# 可选：保存为CSV格式，便于表格分析
qa_df = pd.DataFrame(qa_list)
qa_csv_path = "spark_multi_rag/datas/qa_train.csv"
qa_df.to_csv(qa_csv_path, index=False, encoding="utf-8")
print(f"已保存CSV文件: {qa_csv_path}")

已保存JSON文件: spark_multi_rag/datas/qa_train.json
已保存CSV文件: spark_multi_rag/datas/qa_train.csv
